# 15_MLP_v4_new3 요약

- 모델: MLP 단독
- 피처: new3 기반 축소 버전
- 학습/평가: 단일/간단 실험
- 제출파일: submission_15_MLP_v4_new3.csv


In [7]:
"""
🎯 AutoGluon 딥러닝 Only
========================
- 검증된 18개 피처 사용
- Tree 모델 제외, 딥러닝만
- 자동 하이퍼파라미터 튜닝
"""

import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
import warnings
warnings.filterwarnings('ignore')

# AutoGluon
from autogluon.tabular import TabularPredictor

# ============================================================
# 데이터 로딩 & 전처리
# ============================================================
print("📂 데이터 로딩...")
train_raw = pd.read_csv("../../data/raw/train.csv")
test_raw = pd.read_csv("../../data/raw/test_x.csv")

train_raw["voted_bin"] = (train_raw["voted"] == 2).astype(int)
print(f"Train: {train_raw.shape}, Test: {test_raw.shape}")


def preprocess(df):
    df = df.copy()
    for col in ['education', 'married', 'urban', 'engnat']:
        if col in df.columns:
            df.loc[df[col] == 0, col] = np.nan
    for col in [f"tp{i:02d}" for i in range(1, 11)]:
        if col in df.columns:
            df.loc[df[col] == 0, col] = np.nan
    return df


def build_features(df):
    df = df.copy()
    
    # age_ord
    age_map = {"10s": 1, "20s": 2, "30s": 3, "40s": 4, "50s": 5, "60s": 6, "+70s": 7}
    df["age_ord"] = df["age_group"].map(age_map)
    
    # Interaction
    df['age_edu'] = df['age_ord'] * df['education']
    df['is_teenager'] = (df['age_ord'] == 1).astype(int)
    df['teen_low_edu'] = df['is_teenager'] * (df['education'] <= 2).astype(float)
    
    # WR sum
    wr_cols = [f"wr_{i:02d}" for i in range(1, 14)]
    df['wr_sum'] = df[wr_cols].sum(axis=1)
    
    return df


# 전처리 & 피처 생성
train = preprocess(train_raw)
test = preprocess(test_raw)

train = build_features(train)
test = build_features(test)

# ============================================================
# 피처 선택 (검증된 18개 + 범주형)
# ============================================================
features = [
    # 원본 (5개)
    'age_ord', 'education', 'married', 'urban', 'engnat',
    # Interaction (4개)
    'age_edu', 'is_teenager', 'teen_low_edu', 'wr_sum',
    # TP 상위 (4개)
    'tp07', 'tp06', 'tp04', 'tp03',
    # 범주형 (AutoGluon이 알아서 TE 해줌)
    'age_group', 'race', 'religion', 'gender',
]

# 타겟
label = 'voted_bin'

# 학습 데이터 준비
train_data = train[features + [label]].copy()
test_data = test[features].copy()

print(f"\n📊 피처: {len(features)}개")
print(f"📊 Train: {train_data.shape}, Test: {test_data.shape}")

# ============================================================
# AutoGluon 학습
# ============================================================
print("\n" + "="*60)
print("🚀 AutoGluon 학습 시작 (딥러닝 Only)")
print("="*60)

# 딥러닝 하이퍼파라미터
nn_options = {
    'num_epochs': 100,
    'learning_rate': 1e-3,
    'dropout_prob': 0.3,
}

hyperparameters = {
    'NN_TORCH': nn_options,  # PyTorch MLP
    'FASTAI': {},             # FastAI Neural Net
}

# Tree 모델 제외
excluded = ['GBM', 'CAT', 'XGB', 'RF', 'XT', 'KNN', 'LR']

predictor = TabularPredictor(
    label=label,
    eval_metric='roc_auc',
    path='autogluon_models'
)

predictor.fit(
    train_data=train_data,
    hyperparameters=hyperparameters,
    excluded_model_types=excluded,
    presets='high_quality',      # 핵심 변경
    time_limit=3600 * 6,
    num_bag_folds=10,            # 5 -> 10
    num_bag_sets=2,              # 추가 (시간 괜찮으면)
    num_stack_levels=1,          # 0 -> 1
    verbosity=2
)

# ============================================================
# 결과 확인
# ============================================================
print("\n" + "="*60)
print("📊 학습 결과")
print("="*60)

# 리더보드
leaderboard = predictor.leaderboard(silent=True)
print("\n모델 리더보드:")
print(leaderboard)

# CV 점수
print(f"\n🏆 Best CV Score: {predictor.evaluate(train_data)}")

# ============================================================
# 예측 & 저장
# ============================================================
print("\n" + "="*60)
print("💾 예측 & 저장")
print("="*60)

# 예측
test_pred = predictor.predict_proba(test_data)

# voted_bin=1 (투표 안함)의 확률
if isinstance(test_pred, pd.DataFrame):
    if 1 in test_pred.columns:
        pred_proba = test_pred[1].values
    else:
        pred_proba = test_pred.iloc[:, 1].values
else:
    pred_proba = test_pred

# 저장
sub = pd.DataFrame({
    "index": test_raw["index"] if "index" in test_raw.columns else range(len(test_raw)),
    "voted": pred_proba
})
sub.to_csv("submission_15_MLP_v4_new3.csv", index=False)

print(f"\n💾 저장: submission_15_MLP_v4_new3.csv")
print(f"   예측 범위: [{pred_proba.min():.4f}, {pred_proba.max():.4f}]")
print(f"   예측 평균: {pred_proba.mean():.4f}")

# Feature Importance
print("\n📊 Feature Importance:")
importance = predictor.feature_importance(train_data)
print(importance.head(15))



📂 데이터 로딩...


Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.5.0
Python Version:     3.11.14
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 24.6.0: Mon Jul 14 11:30:29 PDT 2025; root:xnu-11417.140.69~1/RELEASE_ARM64_T6000
CPU Count:          10
Pytorch Version:    2.9.1
CUDA Version:       CUDA is not available
GPU Count:          WARNING: Exception was raised when calculating GPU count (AssertionError)
Memory Avail:       4.80 GB / 16.00 GB (30.0%)
Disk Space Avail:   220.09 GB / 460.43 GB (47.8%)
Presets specified: ['high_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=10, num_bag_sets=2
Note: `save_bag_folds=False`! This will greatly reduce peak disk usage during fit (by ~10x), but runs the risk of an out-of-memory error du

Train: (45532, 79), Test: (11383, 77)

📊 피처: 17개
📊 Train: (45532, 18), Test: (11383, 17)

🚀 AutoGluon 학습 시작 (딥러닝 Only)


Beginning AutoGluon training ... Time limit = 5400s
AutoGluon will save models to "/Users/admin/vote_AI/notebooks/hyemin/autogluon_models/ds_sub_fit/sub_fit_ho"
Train Data Rows:    40472
Train Data Columns: 17
Label Column:       voted_bin
Problem Type:       binary
Preprocessing data ...
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    4936.19 MB
	Train Data (Original)  Memory Usage: 13.74 MB (0.3% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
			Note: Converting 3 features to boolean dtype as they only contain 2 unique values.
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
		Fitting CategoryFeatureGenerator..


📊 학습 결과

모델 리더보드:
                         model  score_val eval_metric  pred_time_val  \
0          WeightedEnsemble_L2   0.769921     roc_auc       1.815608   
1       NeuralNetFastAI_BAG_L1   0.768468     roc_auc       1.077892   
2        NeuralNetTorch_BAG_L1   0.768362     roc_auc       0.732504   
3     WeightedEnsemble_L2_FULL        NaN     roc_auc            NaN   
4   NeuralNetTorch_BAG_L1_FULL        NaN     roc_auc            NaN   
5  NeuralNetFastAI_BAG_L1_FULL        NaN     roc_auc            NaN   

     fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  \
0  179.039301                0.005212           0.271461            2   
1   72.199208                1.077892          72.199208            1   
2  106.568632                0.732504         106.568632            1   
3   22.589385                     NaN           0.271461            2   
4   10.361822                     NaN          10.361822            1   
5   11.956102                     NaN 

Computing feature importance via permutation shuffling for 17 features using 5000 rows with 5 shuffle sets...



🏆 Best CV Score: {'roc_auc': np.float64(0.7796167167899445), 'accuracy': 0.7060309233066854, 'balanced_accuracy': np.float64(0.712470588443225), 'mcc': 0.4251088557939523, 'f1': 0.7054292568058276, 'precision': 0.78024438927024, 'recall': 0.6437063217929151}

💾 예측 & 저장

💾 저장: submission_20_new3_test.csv
   예측 범위: [0.1509, 0.9986]
   예측 평균: 0.5566

📊 Feature Importance:


	6.17s	= Expected runtime (1.23s per shuffle set)
	4.84s	= Actual runtime (Completed 5 of 5 shuffle sets)


              importance    stddev   p_value  n  p99_high   p99_low
education       0.031160  0.006127  0.000170  5  0.043775  0.018546
race            0.025484  0.002198  0.000007  5  0.030009  0.020959
religion        0.013782  0.001424  0.000013  5  0.016714  0.010849
engnat          0.011540  0.000879  0.000004  5  0.013349  0.009731
age_edu         0.010784  0.003428  0.001077  5  0.017843  0.003724
married         0.009932  0.002201  0.000271  5  0.014465  0.005400
age_ord         0.007348  0.001379  0.000142  5  0.010189  0.004508
wr_sum          0.005398  0.001795  0.001273  5  0.009092  0.001703
age_group       0.004696  0.000950  0.000190  5  0.006652  0.002741
tp06            0.003408  0.001795  0.006598  5  0.007104 -0.000287
is_teenager     0.003310  0.001408  0.003136  5  0.006210  0.000410
tp07            0.003297  0.000823  0.000431  5  0.004992  0.001601
tp03            0.003104  0.001292  0.002898  5  0.005765  0.000444
teen_low_edu    0.002509  0.001801  0.017858  5 